In [147]:
# Libraries
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import shuffle

import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools

from keras import layers
from keras import regularizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, concatenate, Input, Conv2D, MaxPooling2D
from keras.optimizers import Adam, Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.np_utils import to_categorical
import tensorflow.keras.optimizers as Optimizer

In [80]:
# Load data
train_dir = r'F:\dataset\DiseaseGrading\Original_Images\Train'
test_dir = r'F:\dataset\DiseaseGrading\Original_Images\Test'
train_cate = pd.read_csv(r'F:\dataset\DiseaseGrading\Groundtruths\Train_Labels.csv')
test_cate = pd.read_csv(r'F:\dataset\DiseaseGrading\Groundtruths\Test_Labels.csv')

In [81]:
train_cate.head()

,Image_name,Retinopathy grade,Risk of macular edema
0,IDRiD_001,3,2
1,IDRiD_002,3,2
2,IDRiD_003,2,2
3,IDRiD_004,3,2
4,IDRiD_005,4,0


In [102]:
np.array(train_cate[train_cate['Image_name']=='IDRiD_001']['Retinopathy grade'])

array([3], dtype=int64)

In [37]:
test_cate.head()

,Image name,Retinopathy grade,Risk of macular edema
0,IDRiD_001,4,0
1,IDRiD_002,4,1
2,IDRiD_003,4,0
3,IDRiD_004,4,0
4,IDRiD_005,4,1


In [166]:
# Get data and labels
train_data = []
train_labels = []

def get_images(image_dir, labels_dir):
    
    for image_file in os.listdir(image_dir):
        image = cv2.imread(image_dir+r'/'+image_file)
        image = cv2.resize(image,(227,227))
        train_data.append(image)
        labels = pd.read_csv(labels_dir)
        label = list(labels[labels['Image_name'] + '.jpg' ==  image_file]['Retinopathy grade'])
        train_labels.append(label)
    
    return shuffle(train_data,train_labels,random_state=7)

In [167]:
train_data, train_labels = get_images(r'F:\dataset\DiseaseGrading\Original_Images\Train', r'F:\dataset\DiseaseGrading\Groundtruths\Train_Labels.csv')

In [168]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [169]:
train_data.shape

(413, 227, 227, 3)

In [170]:
train_labels.shape

(413, 1)

In [171]:
train_labels.reshape(413)

array([2, 0, 2, 4, 1, 0, 2, 3, 2, 2, 0, 3, 2, 4, 0, 0, 0, 3, 4, 3, 1, 3,
       3, 2, 0, 0, 2, 2, 2, 4, 4, 2, 2, 1, 0, 4, 2, 4, 4, 4, 0, 3, 2, 3,
       0, 2, 3, 3, 3, 0, 2, 0, 3, 2, 2, 3, 3, 4, 2, 2, 3, 0, 0, 1, 2, 0,
       0, 0, 4, 2, 0, 0, 2, 2, 0, 0, 0, 4, 2, 3, 1, 0, 2, 4, 3, 2, 0, 0,
       2, 2, 0, 2, 0, 3, 0, 4, 4, 0, 3, 2, 2, 0, 2, 4, 0, 0, 0, 2, 2, 0,
       2, 4, 4, 2, 2, 0, 1, 4, 0, 2, 2, 2, 2, 0, 0, 3, 0, 0, 2, 3, 1, 0,
       4, 0, 4, 0, 2, 2, 2, 2, 0, 2, 2, 3, 2, 3, 1, 2, 4, 2, 2, 0, 0, 0,
       0, 0, 3, 4, 0, 2, 3, 4, 2, 2, 2, 3, 3, 2, 1, 0, 2, 2, 2, 3, 3, 2,
       0, 3, 2, 1, 2, 3, 1, 3, 0, 0, 0, 3, 0, 1, 4, 4, 2, 0, 3, 4, 2, 2,
       0, 3, 3, 0, 2, 3, 0, 2, 0, 2, 2, 2, 3, 3, 0, 0, 2, 0, 4, 0, 0, 3,
       3, 0, 0, 1, 3, 0, 4, 1, 2, 2, 0, 3, 2, 2, 0, 2, 0, 3, 0, 3, 4, 3,
       0, 0, 4, 2, 4, 0, 1, 4, 3, 4, 3, 2, 2, 4, 0, 2, 4, 0, 0, 0, 2, 2,
       2, 2, 2, 0, 0, 1, 2, 0, 3, 0, 4, 0, 2, 3, 3, 3, 0, 3, 2, 2, 2, 2,
       4, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 4, 0, 0,

In [137]:
# for image_file in os.listdir(r'F:\dataset\DiseaseGrading\Original_Images\Train'):
#     print(image_file)

In [161]:
# train_labels = to_categorical(train_labels)

In [162]:
# train_labels

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [144]:
# Build cnn model
model = keras.Sequential([
tf.keras.Input(shape = (227, 227, 3)),
layers.Conv2D (64, (5,5), strides=(1, 1), padding='same', kernel_regularizer = regularizers.l2(0.001), 
               bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope= 1),
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2), padding="same"),

layers.Conv2D (64, (5,5), strides=(1, 1), padding='same', kernel_regularizer = regularizers.l2(0.001), 
               bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1),
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2)),

layers.Conv2D (128, (5,5), strides=(1, 1), padding='same', kernel_regularizer = regularizers.l2(0.001), 
               bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1),
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2)),

layers.Conv2D (256, (3,3), strides=(1, 1), padding='same', kernel_regularizer = regularizers.l2(0.001), 
               bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1),
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2), padding="same"),

layers.Conv2D (512, (3,3), strides=(1, 1), padding='same', kernel_regularizer = regularizers.l2(0.001), 
               bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1), #negative slope 1 is good
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2), padding="same"),

layers.Conv2D (1024, (1,1), strides=(1, 1), padding='same', kernel_regularizer = regularizers.l2(0.001), 
               bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1),
layers.BatchNormalization(),
layers.MaxPooling2D(pool_size=(2, 2), padding="same"),
layers.MaxPooling2D(pool_size=(3, 3), padding="same"),
layers.Reshape((1, 4096)),
layers.Dense(1024, kernel_regularizer = regularizers.l2(0.001), bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1),
layers.Dropout(0.5),
layers.Dense(32, name="my_intermediate_layer", kernel_regularizer = regularizers.l2(0.001), 
             bias_regularizer = regularizers.l2(0.001)),
layers.ReLU(negative_slope = 1),
layers.Dense(5, activation='softmax', name="my_last_layer"),
])

In [145]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 227, 227, 64)      4864      
_________________________________________________________________
re_lu_15 (ReLU)              (None, 227, 227, 64)      0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 227, 227, 64)      256       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 114, 114, 64)      102464    
_________________________________________________________________
re_lu_16 (ReLU)              (None, 114, 114, 64)      0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 114, 114, 64)     

In [187]:
model.compile(optimizer=Optimizer.Adam(lr=0.00005),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [188]:
# Train
trained = model.fit(train_data,train_labels,epochs=30,validation_split=0.10)

Epoch 1/30
12/12 [==============================] - 6s 475ms/step - loss: 3.4634 - accuracy: 0.3181 - val_loss: 3.1041 - val_accuracy: 0.4048
Epoch 2/30
12/12 [==============================] - 4s 369ms/step - loss: 2.5774 - accuracy: 0.3504 - val_loss: 3.9993 - val_accuracy: 0.4524
Epoch 3/30
12/12 [==============================] - 4s 369ms/step - loss: 2.2905 - accuracy: 0.3666 - val_loss: 2.4532 - val_accuracy: 0.4286
Epoch 4/30
12/12 [==============================] - 4s 369ms/step - loss: 2.3875 - accuracy: 0.4205 - val_loss: 2.8394 - val_accuracy: 0.5238
Epoch 5/30
12/12 [==============================] - 4s 370ms/step - loss: 2.3108 - accuracy: 0.3720 - val_loss: 3.6137 - val_accuracy: 0.4286
Epoch 6/30
12/12 [==============================] - 4s 370ms/step - loss: 2.3130 - accuracy: 0.3935 - val_loss: 2.9128 - val_accuracy: 0.4524
Epoch 7/30
12/12 [==============================] - 4s 370ms/step - loss: 2.0884 - accuracy: 0.4205 - val_loss: 1.7207 - val_accuracy: 0.3571
Epoch 

In [189]:
# SVM
model_feat = Model(inputs=model.input,outputs=model.get_layer('my_intermediate_layer').output)
feat_train = model_feat.predict(train_data)
print(feat_train.shape)

(413, 1, 32)


In [190]:
feat_train = feat_train.reshape(413,32)

In [191]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')
svm.fit(feat_train,train_labels)
svm.score(feat_train,train_labels)

0.5907990314769975